In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import tstrippy
from astropy import units as u
from astropy import coordinates as coord
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True

In [49]:
# what is the combination to get time interms of energy?
import sympy as sp

A = sp.Matrix([
    [2, 3, 0],
    [-2, -2, 0],
    [0, -1, 1]
])
b = sp.Matrix([0, 1, 0])
x, y, z = sp.symbols('x y z')
solution = sp.linsolve((A, b), x, y, z)
solution

{(-3/2, 1, 1)}

$$ T_{\mathrm{dyn}}=\frac{GM}{E^{3/2}}$$ 

$$ \delta \theta = \delta \omega \times t$$

$$ \delta \omega = 2\pi \left(\frac{1}{T_1} - \frac{1}{T_2} \right)$$

$$ T_{\mathrm{phase}} = \frac{1}{\delta \omega}$$

$$ T_{1}=\frac{GM}{\left(E+\delta E\right)^{3/2}}$$ 

$$ T_{2}=\frac{GM}{\left(E-\delta E\right)^{3/2}}$$ 

$$ T_\textrm{phase} = \frac{T_1 T_2}{T_1 - T_2} $$

Doing a taylor series and ignoring all terms higher than $\mathcal{O}(2)$

$$ T_\mathrm{Phase} = T_\mathrm{dyn}\left(\frac{E}{3\delta E}\right)

Phase mixing for a cluster WITHOUT a central force and then one with one

In [305]:
GCname = "NGC6715"
clusters=tstrippy.Parsers.baumgardtMWGCs()
mean,cov=clusters.getGCCovarianceMatrix(GCname)
RA,DEC,Rsun,RV,mualpha,mu_delta,Mass,rh_m=mean
# put this in galactocentric coordinates
# put the coordinates on the sky 
skycoords=coord.sky_coordinate.SkyCoord(
    ra = RA * u.deg,
    dec = DEC * u.deg,
    distance = Rsun * u.kpc,
    radial_velocity = RV * u.km / u.s,
    pm_ra_cosdec = mualpha * u.mas / u.yr,
    pm_dec = mu_delta * u.mas / u.yr,)
galactocentric=skycoords.transform_to(coord.Galactocentric)
# extact the galacticentric coordinates
xc,yc,zc = galactocentric.x.value, galactocentric.y.value, galactocentric.z.value
vxc,vyc,vzc = galactocentric.v_x.value, galactocentric.v_y.value, galactocentric.v_z.value

In [317]:
MWparams = tstrippy.Parsers.pouliasis2017pii()
rplum = tstrippy.ergodic.convertHalfMassRadiusToPlummerRadius(rh_m)
plumParams = [MWparams[0],Mass,rplum]


In [307]:
xp,yp,zp,vxp,vyp,vzp=tstrippy.ergodic.isotropicplummer(MWparams[0],Mass,rh_m,int(5e2))

In [308]:
# Find the mixing time of the system
_,_,_,phiMW=tstrippy.potentials.pouliasis2017pii(MWparams, xp+xc, yp+yc, zp+zc)
_,_,_,phiGC=tstrippy.potentials.plummer(plumParams,xp,yp,zp)
kineticEnergy = 0.5 * ((vxp+vxc)**2 + (vyp+vyc)**2 + (vzp+vzc)**2)
totalEnergy = kineticEnergy + phiMW + phiGC
# get the mean and spread of the total energy
meanE = np.mean(totalEnergy)
stdE = np.std(totalEnergy)


In [309]:
# get the dynamical time 
ax,ay,ax,phiGC=tstrippy.potentials.pouliasis2017pii(MWparams, xc, yc, zc)
force = np.sqrt(ax**2 + ay**2 + ax**2)
tdyn = (1/2)*np.sqrt(rc/force[0])
# measure it the other way too 
r = np.sqrt(xc**2 + yc**2 + zc**2)
v = np.sqrt(vxc**2 + vyc**2 + vzc**2)
tdyn2 = r / v
print("force:", tdyn)
print("r/v", tdyn2)


force: 0.03753498752717843
r/v 0.05913709652749826


In [310]:
# get the mixing time
print(GCname)
print(2*np.pi*tdyn)
print(2*np.pi*np.abs(tdyn * (meanE/(3*stdE))))

NGC6715
0.23583928213593655
1.3090224629821419


In [313]:
stdE/meanE

np.float64(-0.06005480904141264)